# Create Heterostructure Example with Three Materials

This notebook demonstrates how to create a heterostructure involving three different materials using a sequential interface creation approach. We first create an interface between **Material 0** and **Material 1**, and then use that interface as a substrate to add a film of **Material 2**.

<h2 style="color:green">Usage</h2>

1. **Set up the notebook and install packages**
2. **Import materials from Standata**
3. **Select and preview materials for the heterostructure**
4. **Build the heterostructure layer by layer with ZSL interface builder**

## Summary

1. **Prepare the Environment:** Set up the notebook and install packages, preview the input materials.
2. **Create Interfaces:** Sequentially create interfaces between the materials.
3. **Visualize:** Preview the materials and resulting interfaces.
4. **Pass to Runtime:** Pass the final heterostructure to the external runtime.

## Notes

1. For more information, see [Introduction](Introduction.ipynb)

<!-- # TODO: use a hashtag-based anchor link to interface creation documentation above -->


## 1. Prepare the Environment
### 1.1. Set up the Notebook

Set the following flags to control the notebook behavior.


In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

# Indices and configurations for the three materials
MATERIAL_0_INDEX = 0
MATERIAL_1_INDEX = 1
MATERIAL_2_INDEX = 2

# Interface parameters
MAX_AREA_01 = 50 # search area for the first interface
MAX_AREA_12 = 200 # search area for the second interface
INTERFACE_01_DISTANCE = 3.0  # in Angstrom
INTERFACE_12_DISTANCE = 3.0  # in Angstrom
FINAL_INTERFACE_VACUUM = 20.0  # in Angstrom

# Configuration for Material 0 (Substrate)
MATERIAL_0_MILLER_INDICES = (0, 0, 1)
MATERIAL_0_THICKNESS = 3  # in atomic layers
MATERIAL_0_VACUUM = 3  # in Angstroms
MATERIAL_0_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
MATERIAL_0_USE_ORTHOGONAL_Z = True

# Configuration for Material 1 (Film 1)
MATERIAL_1_MILLER_INDICES = (0, 0, 1)
MATERIAL_1_THICKNESS = 1  # in atomic layers
MATERIAL_1_VACUUM = 0  # in Angstroms
MATERIAL_1_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
MATERIAL_1_USE_ORTHOGONAL_Z = True

# Configuration for Material 2 (Film 2)
MATERIAL_2_MILLER_INDICES = (0, 0, 1)
MATERIAL_2_THICKNESS = 1  # in atomic layers
MATERIAL_2_VACUUM = 1  # in Angstroms
MATERIAL_2_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
MATERIAL_2_USE_ORTHOGONAL_Z = True

# Set termination pair indices for both interfaces
TERMINATION_PAIR_INDEX_01 = 0
TERMINATION_PAIR_INDEX_12 = 0


### 1.2. Install Packages

The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")


### 1.3. Get Input Materials and Assign `material0`, `material1`, and `material2`

Materials are loaded with `get_materials()`. The first material is assigned as **Material 0**, the second as **Material 1**, and the third as **Material 2**.


In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

material0 = materials[MATERIAL_0_INDEX]

try: 
    material1 = materials[MATERIAL_1_INDEX]
except IndexError:
    print("Please select Material 1. Material 1 is set to Material 0.")
    material1 = material0

try:
    material2 = materials[MATERIAL_2_INDEX]
except IndexError:
    print("Please select Material 2. Material 2 is set to Material 0.")
    material2 = material0


### 1.4. Preview Original Materials

Visualize the three original materials.


In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([material0, material1, material2], repetitions=[3, 3, 1], rotation="0x")


## 2. Create First Interface (Material 0 + Material 1)

### 2.1. Configure Slabs and Select Termination Pair

Set up slab configurations for **Material 0** and **Material 1**, then select terminations for the first interface.


In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

# Slab Configuration for Material 1
material1_slab_configuration = SlabConfiguration(
    bulk=material1,
    miller_indices=MATERIAL_1_MILLER_INDICES,
    thickness=MATERIAL_1_THICKNESS, # in atomic layers
    vacuum=MATERIAL_1_VACUUM, # in Angstroms
    xy_supercell_matrix=MATERIAL_1_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=MATERIAL_1_USE_ORTHOGONAL_Z
)

# Slab Configuration for Material 0 (Substrate)
material0_slab_configuration = SlabConfiguration(
    bulk=material0,
    miller_indices=MATERIAL_0_MILLER_INDICES,
    thickness=MATERIAL_0_THICKNESS, # in atomic layers
    vacuum=MATERIAL_0_VACUUM, # in Angstroms
    xy_supercell_matrix=MATERIAL_0_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=MATERIAL_0_USE_ORTHOGONAL_Z
)

# Get possible terminations for the slabs
material1_slab_terminations = get_terminations(material1_slab_configuration)
material0_slab_terminations = get_terminations(material0_slab_configuration)

# Visualize all possible terminations
material1_slabs = [create_slab(material1_slab_configuration, termination) for termination in material1_slab_terminations]
material0_slabs = [create_slab(material0_slab_configuration, termination) for termination in material0_slab_terminations]

visualize(
    [{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in material1_slabs],
    repetitions=[3, 3, 1],
    rotation="-90x"
)

visualize(
    [{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in material0_slabs],
    repetitions=[3, 3, 1],
    rotation="-90x"
)


### 2.2. Print and Select Termination Pair for First Interface


In [ ]:
from itertools import product

termination_pairs_01 = list(product(material1_slab_terminations, material0_slab_terminations))    
print("Termination Pairs for First Interface (Material1, Material0)")
for idx, termination_pair in enumerate(termination_pairs_01):
    print(f"    {idx}: {termination_pair}")


### 2.3. Select Termination Pair for First Interface


In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_pair_index_01 = TERMINATION_PAIR_INDEX_01

termination_pair_first = termination_pairs_01[termination_pair_index_01]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair_first = await ui_prompt_select_array_element_by_index_pyodide(
            termination_pairs_01,
            element_name="Material1/Material0 termination pair"
        )
    else:
        termination_pair_first = ui_prompt_select_array_element_by_index(
            termination_pairs_01,
            element_name="Material1/Material0 termination pair"
        )


### 2.4. Initialize Interface Configuration for First Interface


In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

material1_termination, material0_termination = termination_pair_first
interface_configuration_01 = InterfaceConfiguration(
    film_configuration=material1_slab_configuration,
    substrate_configuration=material0_slab_configuration,
    film_termination=material1_termination,
    substrate_termination=material0_termination,
    distance=INTERFACE_01_DISTANCE,
    vacuum=FINAL_INTERFACE_VACUUM
)


### 2.5. Set Strain Matching Parameters and Generate First Interface


In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, ZSLStrainMatchingInterfaceBuilderParameters

zsl_strain_matching_parameters_01 = ZSLStrainMatchingParameters(
    max_area=MAX_AREA_01
)

matched_interfaces_builder_01 = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=zsl_strain_matching_parameters_01
    )
)

interfaces_01_sorted = matched_interfaces_builder_01.get_materials(configuration=interface_configuration_01)


### 2.6. Plot and Select First Interface


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces_01_sorted, PLOT_SETTINGS)

# Select the first interface with the lowest strain and smallest number of atoms
interfaces_slice_range_01 = slice(0, 1)
selected_interfaces_01 = interfaces_01_sorted[interfaces_slice_range_01]


### 2.7. Preview the First Interface


In [ ]:
visualize(selected_interfaces_01, repetitions=[3, 3, 1])
visualize(selected_interfaces_01, repetitions=[3, 3, 1], rotation="-90x")

## 3. Create Second Interface (First Interface + Material 2)

### 3.1. Configure Slabs and Select Termination Pair for Second Interface

Now, use the first interface as the substrate to add **Material 2**.


In [ ]:
from mat3ra.made.tools.modify import translate_to_z_level
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

# Update substrate to be the first interface
substrate_second = translate_to_z_level(selected_interfaces_01[0], "top")

# Slab Configuration for Material 2
material2_slab_configuration = SlabConfiguration(
    bulk=material2,
    miller_indices=MATERIAL_2_MILLER_INDICES,
    thickness=MATERIAL_2_THICKNESS, # in atomic layers
    vacuum=MATERIAL_2_VACUUM, # in atomic layers
    xy_supercell_matrix=MATERIAL_2_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=MATERIAL_2_USE_ORTHOGONAL_Z
)

# Slab Configuration for Substrate (First Interface)
substrate_second_slab_configuration = SlabConfiguration(
    bulk=substrate_second,
    miller_indices=(0, 0, 1),  # Z-orientation for the first interface
    thickness=1, # One unit cell thick
    vacuum=0,  
    xy_supercell_matrix=MATERIAL_0_XY_SUPERCELL_MATRIX,  # Adjust if necessary
    use_orthogonal_z=MATERIAL_0_USE_ORTHOGONAL_Z
)

# Get possible terminations for the second interface slabs
material2_slab_terminations = get_terminations(material2_slab_configuration)
substrate_second_slab_terminations = get_terminations(substrate_second_slab_configuration)

# Visualize all possible terminations for Material 2 and Substrate (First Interface)
material2_slabs = [create_slab(material2_slab_configuration, termination) for termination in material2_slab_terminations]
substrate_second_slabs = [create_slab(substrate_second_slab_configuration, termination) for termination in substrate_second_slab_terminations]

visualize(
    [{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in material2_slabs],
    repetitions=[3, 3, 1],
    rotation="-90x"
)

visualize(
    [{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in substrate_second_slabs],
    repetitions=[3, 3, 1],
    rotation="-90x"
)

### 3.2. Print and Select Termination Pair for Second Interface


In [ ]:
termination_pairs_12 = list(product(material2_slab_terminations, substrate_second_slab_terminations))    
print("Termination Pairs for Second Interface (Material2, First Interface Substrate)")
for idx, termination_pair in enumerate(termination_pairs_12):
    print(f"    {idx}: {termination_pair}")

### 3.3. Select Termination Pair for Second Interface


In [ ]:
termination_pair_index_12 = TERMINATION_PAIR_INDEX_12

termination_pair_second = termination_pairs_12[termination_pair_index_12]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair_second = await ui_prompt_select_array_element_by_index_pyodide(
            termination_pairs_12,
            element_name="Material2/First Interface termination pair"
        )
    else:
        termination_pair_second = ui_prompt_select_array_element_by_index(
            termination_pairs_12,
            element_name="Material2/First Interface termination pair"
        )

### 3.4. Initialize Interface Configuration for Second Interface


In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

material2_termination, substrate_second_termination = termination_pair_second
interface_configuration_12 = InterfaceConfiguration(
    film_configuration=material2_slab_configuration,
    substrate_configuration=substrate_second_slab_configuration,
    film_termination=material2_termination,
    substrate_termination=substrate_second_termination,
    distance=INTERFACE_12_DISTANCE,
    vacuum=FINAL_INTERFACE_VACUUM
)

### 3.5. Set Strain Matching Parameters and Generate Second Interface


In [ ]:
zsl_strain_matching_parameters_12 = ZSLStrainMatchingParameters(
    max_area=MAX_AREA_12
)

matched_interfaces_builder_12 = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=zsl_strain_matching_parameters_12
    )
)

interfaces_12_sorted = matched_interfaces_builder_12.get_materials(configuration=interface_configuration_12)

### 3.6. Plot and Select Second Interface


In [ ]:
plot_strain_vs_atoms(interfaces_12_sorted, PLOT_SETTINGS)

# Select the first interface with the lowest strain and smallest number of atoms
interfaces_slice_range_12 = slice(0, 1)
selected_interfaces_12 = interfaces_12_sorted[interfaces_slice_range_12]

### 3.7. Preview the Second Interface (Final Heterostructure)


In [ ]:
visualize(selected_interfaces_12, repetitions=[3, 3, 1])
visualize(selected_interfaces_12, repetitions=[3, 3, 1], rotation="-90x")

## 4. Preview the Final Heterostructure


In [ ]:
visualize(selected_interfaces_12, repetitions=[3, 3, 1], title="Final Heterostructure (First Interface + Material2)")
visualize(selected_interfaces_12, repetitions=[3, 3, 1], rotation="-90x", title="Final Heterostructure (First Interface + Material2) Rotated")

## 5. Pass the Final Heterostructure to the Outside Runtime

Pass the resulting heterostructure with an adjusted name to `set_materials()`.


In [ ]:
from utils.jupyterlite import set_materials

final_heterostructure = selected_interfaces_12[0]
final_heterostructure.name = f"{material0.name} - {material1.name} - {material2.name} - Heterostructure"

set_materials(final_heterostructure)